In [2]:
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [4]:
user_input = input("Enter the location to search on Google Maps: ")

Enter the location to search on Google Maps:  Taft


In [409]:

# Initialize the browser
browser = webdriver.Chrome()

# Navigate to Google Maps
browser.get('https://www.google.com/maps')

# Wait for the page to load
time.sleep(3)

# Find the search bar and enter the user input
search_bar = browser.find_element(By.XPATH, '//*[@id="searchboxinput"]')
search_bar.send_keys(user_input)

# Wait for the suggestions to load and appear
time.sleep(2)

# Find and click the first suggestion
try:
    first_suggestion = browser.find_element(By.CLASS_NAME, 'DgCNMb')
    first_suggestion.click()
except:
    time.sleep(1)
# Wait for the place to load and appear
time.sleep(2)

# store the address for future use (if there is)

loc_address = browser.find_element(By.CSS_SELECTOR, 'span.DkEaL').text

# Click Restaurant Option
restaurants_option = browser.find_element(By.XPATH, '//button[@aria-label="Restaurants"]')
restaurants_option.click()

# Wait for all restaurants to load and appear
time.sleep(2)

#filter 3.5*only
rating = browser.find_element(By.XPATH, '//*[@id="assistive-chips"]/div/div/div/div[1]/div/div/div/div/div[2]/div[2]/div[1]/button/div/span[2]')
rating.click()

time.sleep(2)

rating = browser.find_element(By.XPATH, '//*[@id="action-menu"]/div[5]')
rating.click()
time.sleep(2)

#Zoom out function so that the search area button works
zoom_out = browser.find_element(By.XPATH, '//*[@id="widget-zoom-out"]/div')
zoom_out.click()

time.sleep(1)

#Search Area button
search_area = browser.find_element(By.XPATH, '//*[@id="search-this-area"]/div/button/span')
search_area.click()

time.sleep(2)

# Initialize a dictionary to store restaurant details
restaurants_dict = {}

# Scroll and gather restaurant details
while True:
    # Get all restaurant elements
    restaurant_list = browser.find_elements(By.CLASS_NAME, 'Nv2PK')
    action = ActionChains(browser)
    scroll_origin = ScrollOrigin.from_element(restaurant_list[len(restaurant_list)-1])
    action.scroll_from_origin(scroll_origin, 0, 2000).perform()
    time.sleep(1)
    new_restaurant_list = browser.find_elements(By.CLASS_NAME, 'Nv2PK')
    if len(new_restaurant_list) == len(restaurant_list):
        break

#clicks each element then extracts corresponding info
for i in range(len(restaurant_list)):
    action = ActionChains(browser)
    scroll_origin = ScrollOrigin.from_element(restaurant_list[i])
    action.scroll_from_origin(scroll_origin, 0, 150).perform()
    time.sleep(1)
    action.move_to_element(restaurant_list[i]).perform()
    restaurant_list[i].click()
    time.sleep(3)
    
    try:
        review_element = WebDriverWait(browser, 5).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'span[aria-label*="review"]'))
        )
            # Extract the number of reviews from the text
    except:
        continue
        
        # Extract the number of reviews from the text
    num_of_reviews = int(review_element.text[1:-1].replace(',', ''))
    if num_of_reviews <= 65:
        continue
        
    #Restaurant Name
    restaurant_name = browser.find_element(By.CSS_SELECTOR, '.a5H0ec').find_element(By.XPATH, './..').text
    
    #Restaurant Rating
    restaurant_rating_element = WebDriverWait(browser, 2).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]'))
    )
    restaurant_rating = restaurant_rating_element.text
    #Restaurant Address
    restaurant_address = browser.find_element(By.CLASS_NAME, 'Io6YTe.fontBodyMedium.kR99db').text

    #Restaurant Contact Number
    try: 
        restaurant_contact = browser.find_element(By.CSS_SELECTOR, 'button[data-item-id^="phone:"] .Io6YTe.fontBodyMedium.kR99db').text
    except: 
        restaurant_contact = "N/A"

    #Restaurant Hours
    try:
        restaurant_hours = browser.find_element(By.CSS_SELECTOR, 'span.ZDu9vd').text
        if "Closes" in restaurant_hours:
            restaurant_hours = restaurant_hours.replace('\u202f', ' ').strip()[7:]
            
    except:
        try:
            restaurant_hours = browser.find_element(By.CSS_SELECTOR, 'div[aria-label="Open 24 hours"]').text
        except:
            restaurant_hours = "N/A"
    restaurants_dict[restaurant_name] = {'Rating': restaurant_rating, 'Address': restaurant_address, 'Contact': restaurant_contact, 'Operating Hours': restaurant_hours}

#Go back to the previous page (location view)
back = browser.find_element(By.XPATH, '//*[@id="searchbox"]/div[2]/div/button/span')
back.click()

# Wait for the page to navigate back
time.sleep(2)

# Explicitly wait for the "Things to do" option to be clickable
things_to_do_option = browser.find_element(By.XPATH, '//*[@id="assistive-chips"]/div/div/div/div[1]/div/div/div/div/div[2]/div[2]/div[3]/button/div/span[1]')
things_to_do_option.click()

# Wait for all things to do to appear
time.sleep(2)

#filter 3.5*only
rating = browser.find_element(By.XPATH, '//*[@id="assistive-chips"]/div/div/div/div[1]/div/div/div/div/div[2]/div[2]/div[1]/button/div/span[2]')
rating.click()

time.sleep(2)

rating = browser.find_element(By.XPATH, '//*[@id="action-menu"]/div[5]')
rating.click()
time.sleep(3)

#Zoom out function so that the search area button works
zoom_out = browser.find_element(By.XPATH, '//*[@id="widget-zoom-out"]/div')
zoom_out.click()

time.sleep(2)

#Search Area button
search_area = browser.find_element(By.XPATH, '//*[@id="search-this-area"]/div/button/span')
search_area.click()

time.sleep(2)

# Initialize a dictionary to store things to do details
things_to_do_dict = {}

# Scroll and gather things to do details
while True:
    # Get all things to do elements
    things_to_do_list = browser.find_elements(By.CLASS_NAME, 'Nv2PK')
    action = ActionChains(browser)
    scroll_origin = ScrollOrigin.from_element(things_to_do_list[len(things_to_do_list)-1])
    action.scroll_from_origin(scroll_origin, 0, 2000).perform()
    time.sleep(3)
    new_things_to_do_list = browser.find_elements(By.CLASS_NAME, 'Nv2PK')
    if len(new_things_to_do_list) == len(things_to_do_list):
        break

action = ActionChains(browser)
#clicks each element then extracts info
for i in range(len(things_to_do_list)):
    action = ActionChains(browser)
    scroll_origin = ScrollOrigin.from_element(things_to_do_list[i])
    action.scroll_from_origin(scroll_origin, 0, 200).perform()
    time.sleep(1)
    action.move_to_element(things_to_do_list[i]).perform()
    things_to_do_list[i].click()
    time.sleep(3)

    #Activity Reviews
    try:
        review_element = WebDriverWait(browser, 5).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'span[aria-label*="review"]'))
            )
            # Extract the number of reviews from the text
    except:
        continue
        
        # Extract the number of reviews from the text
    num_of_reviews = int(review_element.text[1:-1].replace(',', ''))
    if num_of_reviews <= 100:
        continue
        
    #Activity Name
    activity_name = browser.find_element(By.CSS_SELECTOR, '.a5H0ec').find_element(By.XPATH, './..').text

    #Activity Rating
    activity_rating_element = WebDriverWait(browser, 2).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]'))
        )
    activity_rating = activity_rating_element.text
    #Activity Address
    activity_address = browser.find_element(By.CLASS_NAME, 'Io6YTe.fontBodyMedium.kR99db').text

    #Activity Contact
    try: 
        activity_contact = browser.find_element(By.CSS_SELECTOR, 'button[data-item-id^="phone:"] .Io6YTe.fontBodyMedium.kR99db').text
    except: 
        activity_contact = "N/A"

    #Activity Hours
    try:
        activity_hours = browser.find_element(By.CSS_SELECTOR, 'span.ZDu9vd').text
        if "Closes" in activity_hours:
            activity_hours = activity_hours.replace('\u202f', ' ').strip()[7:]
            
    except:
        try:
            activity_hours = browser.find_element(By.CSS_SELECTOR, 'div[aria-label="Open 24 hours"]').text
        except:
            activity_hours = "N/A"

    things_to_do_dict[activity_name] = {'Rating': activity_rating, 'Address': activity_address, 'Contact': activity_contact, 'Operating Hours': activity_hours}

browser.back()
browser.back()

time.sleep(2)
search_bar.clear()

# Type in the search bar to search for shopping malls
if loc_address.count(',') > 1:
    search_bar.send_keys("Shopping Malls near " + loc_address)

else:
    search_bar.send_keys("Shopping Malls near " + user_input)

time.sleep(1)

# Press the search button
search = browser.find_element(By.XPATH, '//*[@id="searchbox-searchbutton"]/span')
search.click()

time.sleep(3)

#filter 3.5*only
rating = browser.find_element(By.XPATH, '//*[@id="assistive-chips"]/div/div/div/div[1]/div/div/div/div/div[2]/div[2]/div[1]/button/div/span[2]')
rating.click()

time.sleep(2)

rating = browser.find_element(By.XPATH, '//*[@id="action-menu"]/div[5]')
rating.click()
time.sleep(2)

# Wait for the results to load
time.sleep(2)

# Initialize a dictionary to store shopping mall details
shopping_mall_dict = {}

# Scroll and gather things to do details
while True:
    # Get all mall elements
    shopping_mall_list = browser.find_elements(By.CLASS_NAME, 'Nv2PK')
    action = ActionChains(browser)
    scroll_origin = ScrollOrigin.from_element(shopping_mall_list[len(shopping_mall_list)-1])
    action.scroll_from_origin(scroll_origin, 0, 2000).perform()
    time.sleep(1)
    new_shopping_mall_list = browser.find_elements(By.CLASS_NAME, 'Nv2PK')
    if len(new_shopping_mall_list) == len(shopping_mall_list):
        break

action = ActionChains(browser)
#clicks each element then extracts info
for i in range(len(shopping_mall_list)):
    action = ActionChains(browser)
    scroll_origin = ScrollOrigin.from_element(shopping_mall_list[i])
    action.move_to_element(shopping_mall_list[i]).perform()
    shopping_mall_list[i].click()
    time.sleep(2)

    #Shopping mall Reviews
    try:
        review_element = WebDriverWait(browser, 5).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'span[aria-label*="review"]'))
            )
            # Extract the number of reviews from the text
    except:
        continue
        
        # Extract the number of reviews from the text
    num_of_reviews = int(review_element.text[1:-1].replace(',', ''))
    if num_of_reviews <= 500:
        continue
    #Shopping Mall Name
    shopping_mall_name = browser.find_element(By.CSS_SELECTOR, '.a5H0ec').find_element(By.XPATH, './..').text

    #Shopping Mall Rating
    shopping_mall_rating_element = WebDriverWait(browser, 5).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]'))
    )
    shopping_mall_rating = shopping_mall_rating_element.text
    #Shopping Mall Address
    shopping_mall_address = browser.find_element(By.CLASS_NAME, 'Io6YTe.fontBodyMedium.kR99db').text

    #Shopping Mall Contact
    try: 
        shopping_mall_contact = browser.find_element(By.CSS_SELECTOR, 'button[data-item-id^="phone:"] .Io6YTe.fontBodyMedium.kR99db').text
    except: 
        shopping_mall_contact = "N/A"

    #Shopping Mall Hours
    try:
        shopping_mall_hours = browser.find_element(By.CSS_SELECTOR, 'span.ZDu9vd').text
        if "Closes" in shopping_mall_hours:
            shopping_mall_hours = activity_hours.replace('\u202f', ' ').strip()[7:]
            
    except:
        try:
            shopping_mall_hours = browser.find_element(By.CSS_SELECTOR, 'div[aria-label="Open 24 hours"]').text
        except:
            shopping_mall_hours = "N/A"

    shopping_mall_dict[shopping_mall_name] = {'Rating': shopping_mall_rating, 'Address': shopping_mall_address, 'Contact': shopping_mall_contact, 'Operating Hours': shopping_mall_hours}

# Print the gathered restaurant details
print(restaurants_dict)

# Print the gathered things to do details
print(things_to_do_dict)

# Close the browser
browser.quit()


{'Habanero Kitchen Bar': {'Rating': '4.6', 'Address': 'Araneta Center, Cubao Expo, General Romulo Ave, Cubao, Quezon City, 1109', 'Contact': '0977 206 5489', 'Operating Hours': 'Closes 10 PM'}, 'The Porch by Casa Verde': {'Rating': '4.4', 'Address': '113 Anonas Ext, Diliman, Quezon City, Metro Manila', 'Contact': '(02) 8952 7178', 'Operating Hours': 'Closes 10 PM'}, 'a mano Araneta City (Quezon City)': {'Rating': '4.8', 'Address': 'Ground Floor, Gateway Mall 2, 28 General Araneta, Cubao, Quezon City, 1109 Metro Manila', 'Contact': '0917 888 1415', 'Operating Hours': 'N/A'}, "TACIO'S": {'Rating': '4.2', 'Address': 'New Frontier Theatre, General Roxas Ave, Cubao, Quezon City, 1109 Metro Manila', 'Contact': '0916 564 3675', 'Operating Hours': 'Closes 1 AM'}, "Bellini's": {'Rating': '4.3', 'Address': 'Cubao Expo, 3 General Romulo Ave, Cubao, Quezon City, 1109 Metro Manila', 'Contact': '(02) 8913 2550', 'Operating Hours': 'Closes 9:30 PM'}, 'Friuli Trattoria': {'Rating': '4.4', 'Address': '

In [269]:
# Convert the restaurants dictionary to a DataFrame and add a new column for type
df_restaurants = pd.DataFrame.from_dict(restaurants_dict, orient='index')
df_restaurants.reset_index(inplace=True)
df_restaurants.rename(columns={'index': 'Name'}, inplace=True)
df_restaurants['Type'] = 'Restaurant'

# Convert the things to do dictionary to a DataFrame and add a new column for type
df_things_to_do = pd.DataFrame.from_dict(things_to_do_dict, orient='index')
df_things_to_do.reset_index(inplace=True)
df_things_to_do.rename(columns={'index': 'Name'}, inplace=True)
df_things_to_do['Type'] = 'Attraction'

df_restaurants
# # Concatenate the two DataFrames
# df_combined = pd.concat([df_restaurants, df_things_to_do], ignore_index=True)

# # Display the combined DataFrame
# df_combined
# print(restaurants_dict)

,Name,Rating,Address,Contact,Operating Hours,Type
0,Kanto Freestyle Breakfast La Salle,4.4,"2466 Leon Guinto St, Malate, Manila, 1004 Metr...",0919 478 8055,Open 24 hours,Restaurant
1,El Poco Cantina,4.7,"945 Estrada St, Malate, Manila, 1004 Metro Manila",0920 980 0945,Closes 10:30 PM,Restaurant
2,Chomp Chomp,4.3,"2nd Floor, Bellago Residence, 2450 Leon Guinto...",(02) 8775 7028,Closes 10 PM,Restaurant
3,Ababu,3.9,"931 Estrada St, Malate, Manila, 1004 Metro Manila",N/A,Closes 11:30 PM,Restaurant
4,Zark's Burgers,4.2,"2464 Taft Ave, Malate, Manila, 1000 Metro Manila",(02) 3392 1211,Closes 10 PM,Restaurant
5,Angrydobo,3.7,"2456, 1004 Taft Ave, Malate, Manila, 1004 Metr...",0968 619 4971,Closes 9 PM,Restaurant
6,Tinuhog Ni Benny Barbecue House,4.4,"879 Dagonoy St, Malate, Manila, 1004 Metro Manila",(02) 8253 1072,Closes 2:30 AM,Restaurant
7,Arabic house,4.9,"912 P.Ocampo St., Malate, Manila, 1104 Metro M...",0915 633 1309,N/A,Restaurant
8,Sarang Mart and Restaurant,4.3,"Mervin Terraces, 2nd floor bldg P., 980 P.Ocam...",0915 142 1506,Closes 10:30 PM,Restaurant
9,Tapa King Taft Avenue,3.9,"1st Flr., Commercial Unit No. 5, One Archers P...",(02) 8888 8272,Open 24 hours,Restaurant


In [519]:
display(df_things_to_do)

,Name,Rating,Address,Contact,Operating Hours,Type
0,Upside Down Museum,4.3,"Boom Na Boom Grounds CCP Complex, Roxas Blvd, ...",(02) 8551 5530,N/A,Attraction
1,Rajah Sulayman Park,4.2,"HX9M+F9H, Roxas Blvd, Malate, Manila, 1004 Met...",N/A,Open 24 hours,Attraction
2,Rizal Park,4.5,"Ermita, Manila, 1000 Metro Manila",N/A,Closes 8 PM,Attraction
3,National Museum of Fine Arts,4.7,"Padre Burgos Ave, Ermita, Manila, 1000 Metro M...",(02) 8527 1215,soon ⋅ 6 PM ⋅ Opens 9 AM Fri,Attraction
4,Manila Baywalk Dolomite Beach,4.2,"Roxas Blvd, Malate, Manila, 1004 Metro Manila",N/A,Closed ⋅ Opens 6 AM Fri,Attraction
5,National Museum of Natural History,4.7,"Teodoro F. Valencia Cir, Ermita, Manila, 1000 ...",(02) 8298 1100,soon ⋅ 6 PM ⋅ Opens 9 AM Fri,Attraction
6,Manila Zoo,3.7,"Adriatico St, Malate, Manila, 1004 Metro Manila",0909 836 6911,soon ⋅ 6 PM ⋅ Opens 9 AM Fri,Attraction
7,Paco Park,4.5,"HXJQ+99M, Belen, Paco, Manila, Metro Manila",N/A,Closed ⋅ Opens 6 AM Fri,Attraction
8,Rizal Park - North,4.6,"210 Taft Ave, Ermita, Manila, 1000 Metro Manila",N/A,Open 24 hours,Attraction
9,National Museum of Anthropology,4.7,"HXPJ+3C6, P. Burgos Drive Rizal Park, Teodoro ...",(02) 8298 1100,soon ⋅ 6 PM ⋅ Opens 9 AM Fri,Attraction


In [266]:
df_things_to_do

,Name,Rating,Address,Contact,Operating Hours,Type
0,Fort Santiago,4.6,"HXVC+M5V, Intramuros, Manila, 1002 Metro Manila",(02) 8527 3155,Closes 8 PM,Attraction
1,Manila Ocean Park,4.2,"Quirino Grandstand, 666 Behind, Ermita, Manila...",(02) 7238 6188,Closes 6 PM,Attraction
2,Rizal Park,4.5,"Ermita, Manila, 1000 Metro Manila",N/A,Closes 8 PM,Attraction
3,National Museum of Fine Arts,4.7,"Padre Burgos Ave, Ermita, Manila, 1000 Metro M...",(02) 8527 1215,Closes 6 PM,Attraction
4,Ayala Museum,4.6,"Greenbelt Park, Makati Avenue, corner Dela Ros...",0917 834 3845,Closes 6 PM,Attraction
...,...,...,...,...,...,...
74,Cheeky Monkeys,4.4,"2nd Floor, Met Live Mall, Diosdado Macapagal B...",0968 866 2911,Closes 8 PM,Attraction
75,Union Church of Manila - Makati,4.8,"123 Legazpi Street, Legazpi Village, Makati, 1...",(02) 8812 5609,N/A,Attraction
76,Robinsons Place Manila,4.6,"Pedro Gil, cor Adriatico St, Ermita, Manila, 1...",(02) 8310 3333,Closes 10 PM,Attraction
77,Philippine Air Force Aerospace Museum,4.3,"Pasay, 1309 Metro Manila",N/A,Temporarily closed,Attraction


In [475]:
# Convert the restaurants dictionary to a DataFrame and add a new column for type
df_restaurants = pd.DataFrame.from_dict(restaurants_dict, orient='index')
df_restaurants.reset_index(inplace=True)
df_restaurants.rename(columns={'index': 'Name'}, inplace=True)
df_restaurants['Type'] = 'Restaurant'

#asks if there are specific stuff where the person wants to visit and how many restos to visit
#if not given enough specific locs, it will do a random search
no_restaurant = int(input("How many food spots do you want to visit?"))
while no_restaurant > len(df_restaurants):
    print("Not enough locations to support the number")
    no_restaurant = int(input("How many food spots do you want to visit?"))
list_restaurants = []
specific_restaurant = ''
while  len(list_restaurants) < no_restaurant:
    specific_restaurant = input('Any specific restaurants to visit? (quit if done)')
    if specific_restaurant.upper() == 'QUIT':
        break
    if not df_restaurants[df_restaurants['Name'] ==  specific_restaurant].empty:
        row = df_restaurants[df_restaurants["Name"] == specific_restaurant].to_dict()
        key = list(row['Name'].keys())[0]
        restaurant = "Name: " + row['Name'][key] + "\nRating: " + row['Rating'][key] + "\nAddress: " + row['Address'][key] + "\nContact: " + row['Contact'][key] + "\nHours: " + row['\nOperating Hours'][key]
        list_restaurants.append(restaurant)
        df_restaurants = df_restaurants.drop(index = key)
    else:
        print('Location is not in the dataframe')
#subtracts the number of specific locations to the number of locs wanted
no_restaurant = no_restaurant - len(list_restaurants)
rows = df_restaurants.sample(n=no_restaurant)

for key in rows.get('Name').keys():
    restaurant = "Name: " + rows['Name'][key] + "\nRating: " + rows['Rating'][key] + "\nAddress: " + rows['Address'][key] + "\nContact: " + rows['Contact'][key] + "\nHours: " + rows['Operating Hours'][key]
    list_restaurants.append(restaurant)
    
list_restaurants

How many food spots do you want to visit? 2
Any specific restaurants to visit? (quit if done) quit


['\nName: Eat Fresh Katipunan - Southbound\nRating: 4.4\nAddress: Blueridge Village, 209 Katipunan Ave, Project 4, Quezon City, 1109 Metro Manila\nContact: 0977 042 5885\nHours: Closes 11 PM',
 "\nName: TACIO'S\nRating: 4.2\nAddress: New Frontier Theatre, General Roxas Ave, Cubao, Quezon City, 1109 Metro Manila\nContact: 0916 564 3675\nHours: Closes 1 AM"]

In [477]:
# Convert the things to do dictionary to a DataFrame and add a new column for type
df_things_to_do = pd.DataFrame.from_dict(things_to_do_dict, orient='index')
df_things_to_do.reset_index(inplace=True)
df_things_to_do.rename(columns={'index': 'Name'}, inplace=True)
df_things_to_do['Type'] = 'Attraction'

#asks if there are specific stuff where the person wants to visit and how many restos to visit
#if not given enough specific locs, it will do a random search
no_attraction = int(input("How many attractions do you want to visit?"))
while no_attraction > len(df_things_to_do):
    print("Not enough locations to support the number")
    no_attraction = int(input("How many food spots do you want to visit?"))
list_attractions = []
specific_attraction = ''
while  len(list_attractions) < no_attraction:
    specific_attraction = input('Any specific attractions to visit? (quit if done)')
    if specific_attraction.upper() == 'QUIT':
        break
    if not df_things_to_do[df_things_to_do['Name'] ==  specific_attraction].empty:
        row = df_things_to_do[df_things_to_do["Name"] == specific_attraction].to_dict()
        key = list(row['Name'].keys())[0]
        attraction = "Name: " + row['Name'][key] + "\nRating: " + row['Rating'][key] + "\nAddress: " + row['Address'][key] + "\nContact: " + row['Contact'][key] + "\nHours: " + row['\nOperating Hours'][key]
        list_attractions.append(attraction)
        df_things_to_do = df_things_to_do.drop(index = key)
    else:
        print('Location is not in the dataframe')
#subtracts the number of specific locations to the number of locs wanted
no_attraction = no_attraction - len(list_attractions)
rows = df_things_to_do.sample(n=no_attraction)

for key in rows.get('Name').keys():
    attraction = "Name: " + rows['Name'][key] + "\nRating: " + rows['Rating'][key] + "\nAddress: " + rows['Address'][key] + "\nContact: " + rows['Contact'][key] + "\nHours: " + rows['Operating Hours'][key]
    list_attractions.append(attraction)
    
list_attractions

How many attractions do you want to visit? 2
Any specific attractions to visit? (quit if done) quit


['\nName: San Isidro Labrador Parish Church - 44, Pasay City (Archdiocese of Manila)\nRating: 4.7\nAddress: 1830 Taft Ave, Pasay, Metro Manila\nContact: (02) 8525 1188\nHours: N/A',
 '\nName: Robinsons Place Manila\nRating: 4.6\nAddress: Pedro Gil, cor Adriatico St, Ermita, Manila, 1000 Metro Manila\nContact: (02) 8310 3333\nHours: Closes 10 PM']

In [479]:
#initializes the dataframe
df_shopping_mall = pd.DataFrame.from_dict(shopping_mall_dict, orient='index')
df_shopping_mall.reset_index(inplace=True)
df_shopping_mall.rename(columns={'index': 'Name'}, inplace=True)
df_shopping_mall['Type'] = 'Shopping'
df_shopping_mall

#asks if there are specific stuff where the person wants to visit and how many malls to visit
#if not given enough specific locs, it will do a random search
no_mall = int(input("How many shopping malls do you want to visit?"))
while no_mall > len(df_shopping_mall):
    print("Not enough locations to support the number")
    no_mall = int(input("How many shopping malls do you want to visit?"))
list_shopping_mall = []
specific_mall = ''
while  len(list_shopping_mall) < no_mall:
    specific_mall = input('Any specific malls to visit? (quit if done)')
    if specific_mall.upper() == 'QUIT':
        break
    if not df_shopping_mall[df_shopping_mall['Name'] ==  specific_mall].empty:
        row = df_shopping_mall[df_shopping_mall["Name"] == specific_mall].to_dict()
        key = list(row['Name'].keys())[0]
        mall = "Name: " + row['Name'][key] + "\nRating: " + row['Rating'][key] + "\nAddress: " + row['Address'][key] + "\nContact: " + row['Contact'][key] + "\nHours: " + row['\nOperating Hours'][key]
        list_shopping_mall.append(mall)
        df_shopping_mall = df_shopping_mall.drop(index = key)
    else:
        print('Location is not in the dataframe')
#subtracts the number of specific locations to the number of locs wanted
no_mall = no_mall - len(list_shopping_mall)
rows = df_shopping_mall.sample(n=no_mall)

for key in rows.get('Name').keys():
    mall = "Name: " + rows['Name'][key] + "\nRating: " + rows['Rating'][key] + "\nAddress: " + rows['Address'][key] + "\nContact: " + rows['Contact'][key] + "\nHours: " + rows['Operating Hours'][key]
    list_shopping_mall.append(mall)
    
list_shopping_mall
#this list_full is what we input into the chat gpt api

How many shopping malls do you want to visit? 1
Any specific malls to visit? (quit if done) quit


['\nName: Southgate Mall\nRating: 4.0\nAddress: 2258 Chino Roces Ave, Makati, Metro Manila\nContact: 23372031\nHours: ⋅ Closes 9 PM']

In [509]:
time1 = input("What time does the trip start? (00:00 24 hour format)")
time2 = input("What time does the trip end? 00:00 24 hour format)")
print("Create a one-day itinerary lasting from " +time1+ " to " +time2+ " using all of the locations within the following lists of places:\n\n"
      "Restaurants:\n\n" + "\n\n".join(list_restaurants) + "\n\n"
"Attractions:\n\n" + "\n\n".join(list_attractions) + "\n\n"
"Shopping Malls:\n\n" + "\n\n".join(list_shopping_mall) + "\n\n")

What time does the trip start? (00:00 24 hour format) 10000
What time does the trip end? 00:00 24 hour format) 10000


Create a one-day itinerary lasting from 10000 to 10000 using all of the locations within the following lists of places:

Restaurants:


Name: Eat Fresh Katipunan - Southbound
Rating: 4.4
Address: Blueridge Village, 209 Katipunan Ave, Project 4, Quezon City, 1109 Metro Manila
Contact: 0977 042 5885
Hours: Closes 11 PM


Name: TACIO'S
Rating: 4.2
Address: New Frontier Theatre, General Roxas Ave, Cubao, Quezon City, 1109 Metro Manila
Contact: 0916 564 3675
Hours: Closes 1 AM

Attractions:


Name: San Isidro Labrador Parish Church - 44, Pasay City (Archdiocese of Manila)
Rating: 4.7
Address: 1830 Taft Ave, Pasay, Metro Manila
Contact: (02) 8525 1188
Hours: N/A


Name: Robinsons Place Manila
Rating: 4.6
Address: Pedro Gil, cor Adriatico St, Ermita, Manila, 1000 Metro Manila
Contact: (02) 8310 3333
Hours: Closes 10 PM

Shopping Malls:


Name: Southgate Mall
Rating: 4.0
Address: 2258 Chino Roces Ave, Makati, Metro Manila
Contact: 23372031
Hours: ⋅ Closes 9 PM




In [503]:
pip install openai==0.28

   ---------------------------------------- 0.0/76.5 kB ? eta -:--:--
   ---------------- ----------------------- 30.7/76.5 kB 640.0 kB/s eta 0:00:01
   ---------------------------------------- 76.5/76.5 kB 1.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.35.14
    Uninstalling openai-1.35.14:
      Successfully uninstalled openai-1.35.14
Note: you may need to restart the kernel to use updated packages.


In [515]:
import openai
openai.api_key = ''

def generate_itinerary(restaurants, attractions, shopping_malls, time1, time2):
    prompt = ("Create a one-day itinerary lasting from " +time1+ " to " +time2+ " using all of the locations within the following lists of places:\n\n"
        "Restaurants:\n\n" + "\n\n".join(restaurants) + "\n\n"
        "Attractions:\n\n" + "\n\n".join(attractions) + "\n\n"
        "Shopping Malls:\n\n" + "\n\n".join(shopping_malls) + "\n\n")
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ],
            max_tokens=1000,
            temperature=0.7
        )
    
    return response['choices'][0]['message']['content'].strip()
time1 = input("What time does the trip start? (00:00 24 hour format)")
time2 = input("What time does the trip end? 00:00 24 hour format)")
generate_itinerary(list_restaurants, list_attractions, list_shopping_mall, time1, time2)

What time does the trip start? (00:00 24 hour format) 10:00
What time does the trip end? 00:00 24 hour format) 18:00


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
